In [3]:
import pandas as pd
import numpy as np

In [4]:
df_original=pd.read_csv("D:/coding/Projects/5. User baise analysis/user_journey_raw.csv")

In [5]:
df=df_original.drop("session_id", axis=1)

In [6]:
df

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...
...,...,...,...
9930,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9931,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9932,509095,Annual,Other-Other
9933,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...


#### 1. Cleaning data by group all the user

In [7]:
user_ids=set(df['user_id'])

In [8]:
user_journey=[]
for i in df[df['user_id']==1516]['user_journey']:
    for j in range(len(i)):
        user_journey.append(i[j])

In [9]:
len(user_journey)

932

In [10]:
len(df.groupby("user_id").sum()['user_journey'].iloc[0])

932

In [11]:
pd.merge(df.drop('user_journey',axis=1).drop_duplicates(),df.groupby("user_id").sum().drop('subscription_type',axis=1),on='user_id',how='left')

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,3395,Annual,Other-OtherPricing-Sign up-Sign up-Log in-Log ...
2,10107,Annual,Homepage-HomepageHomepage-Homepage-Homepage-Ho...
3,11145,Monthly,Homepage-Log in-Log in-Log in-Log in-Log inHom...
4,12400,Monthly,Homepage-Career tracks-Career tracks-Career tr...
...,...,...,...
1345,509060,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
1346,509061,Annual,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1347,509085,Annual,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1348,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...


In [12]:
def group_by(data,group_column='user_id',target_column = 'user_journey',sessions='all',count_from='last'):
    
    data
    
    data.drop(target_column,axis=1).drop_duplicates()
    try:
        data.groupby(group_column).sum().drop('subscription_type',axis=1)
    except:
        data.groupby(group_column).sum()
    

In [13]:
pd.DataFrame(columns=df.columns)

,user_id,subscription_type,user_journey


In [14]:
df

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...
...,...,...,...
9930,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9931,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9932,509095,Annual,Other-Other
9933,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...


#### defining a function to combine user_journey and to pick what sessions are required

In [15]:
def group_data(data,group_column='user_id',target_column='target_column',session='all',count_from='last'):
    empty_df=pd.DataFrame(columns=data.columns)
    groups = set(data[group_column])
    
    if session=='all':
        start=0
        end=None
    elif session =="all_except_last":
        start=0
        end=-1
    elif session=="Last":
        start=-session
        end=None
    elif session=="fisrt":
        start=0
        end=session
    
    for group_value in groups:
        
        group_mask=list(data[group_column]==group_value)
        
        group_table=data[group_mask]
        user_journey = "-".join(list(group_table['user_journey'])[start:end])
        new_index=len(empty_df)
        empty_df.loc[new_index] = group_table.iloc[0].copy() # all values except target_column are copied from the first occurence
        empty_df.loc[new_index]['user_journey'] = user_journey
        empty_df.sort_values(by=group_column,ignore_index=True,inplace=True)
        empty_df.reset_index(drop=True,inplace=True)
    return empty_df


In [16]:
clean_data=group_data(df)

C:\Users\phing\AppData\Local\Temp\ipykernel_6548\1384684522.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  empty_df.loc[new_index]['user_journey'] = user_journey
C:\Users\phing\AppData\Local\Temp\ipykernel_6548\1384684522.py:26: Settin

In [17]:
clean_data

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,3395,Annual,Other-Other
2,10107,Annual,Homepage-Homepage
3,11145,Monthly,Homepage-Log in-Log in-Log in-Log in-Log in
4,12400,Monthly,Homepage-Career tracks-Career tracks-Career tr...
...,...,...,...
1345,509060,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
1346,509061,Annual,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1347,509085,Annual,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1348,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...


In [18]:
def remove_pages(data, pages = [], target_column = 'user_journey'):

    df = data.copy()
    pages = set(pages)
    
    if len(pages) == 0:
        return df

    user_journey = list(df[target_column])
    user_journey = [journey.split('-') for journey in user_journey]
    user_journey = [[page for page in journey if page not in pages] for journey in user_journey]
    user_journey = ["-".join(i) for i in user_journey]
    df[target_column] = user_journey
    return df
    

In [19]:
clean_data = remove_pages(clean_data, [])

In [20]:
def remove_page_duplicates(data, target_column = 'user_journey'):
    df = data.copy()
    
    user_journey = list(df[target_column])
    user_journey = [journey.split('-') for journey in user_journey]
    for i in range(len(user_journey)):
        j = 1
        
        while(j < len(user_journey[i])):
            
            if user_journey[i][j-1] == user_journey[i][j]:
                user_journey[i].pop(j)
            
            else:
                j += 1
    
    user_journey = ["-".join(i) for i in user_journey]
    
    df[target_column] = user_journey
    
    
    
    return df

In [21]:
clean_data=remove_page_duplicates(clean_data, target_column = 'user_journey')

In [22]:
clean_data

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Other
1,3395,Annual,Other
2,10107,Annual,Homepage
3,11145,Monthly,Homepage-Log in
4,12400,Monthly,Homepage-Career tracks-Sign up-Log in
...,...,...,...
1345,509060,Annual,Other
1346,509061,Annual,Coupon
1347,509085,Annual,Coupon
1348,509095,Annual,Other


In [23]:
clean_data.to_csv("clean_data.csv",index=False)